<a href="https://colab.research.google.com/github/pollinations/hive/blob/main/notebooks/1%20Text-To-Image/2%20CLIP-Guided%20VQGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://cloudflare-ipfs.com/ipfs/QmUKydjhLKP26P8qT49cvsDYXjmYAShbo2Z95JxatvzwkK" width="300" height="300" />

Input: "*Self-pollination in the style of Ernst Haeckel*"


This model has defined the looks of text-to-image of summer 2021 and has been dominating generative media for quite a while. It works especially well with a style or artist in the text prompt. Another interesting feature is the ability to provide an image from which it starts.

---

It is possible to provide multiple text prompts with weights by providing them like this: 
**```oil painting: 0.5|salvador dali: 0.3|edward much:0.6|robot friend: 1.0|text:-0.5```** *In this case a negative weight next to `text` makes the model avoid text.* 

Credits: [Katherine Crowson](https://twitter.com/RiversHaveWings), [jbuster](https://twitter.com/jbusted1), [thomash](https://twitter.com/pollinations_ai)


[UPD 13.12.2021] Re-order library imports to fix bug that was crashing a lot of runs

[UPD 10.12.2021] Image prompting and extra parameters

[UPD 3.11.2021] Added Gumbel sampling method suggested by Daniel Russ on Discord

[UPD 2.11.2021] Added ruDALLE model

In [ ]:
# Text Prompt
text_input = 'Self-pollination in the style of Ernst Haeckel'  #@param {type: "string"}

# Width in pixels of image to be generated. If you put this too high the GPU may run out of memory. Rather use super-resolution to achieve high resolutions. (advanced)
width = 512 #@param {type: "number"}

# Height in pixels of image to be generated. (see width) (advanced)
height = 512 #@param {type: "number"}

# Apply a (neural) super-resolution step (2 x image width x image height)
super_resolution = False #@param {type: "boolean"}

# Image prompt. Leave blank if you want to use only text
image_file = '' #@param {type: "string"}

# Step size - how fast to try and optimize the image. Between 0 and 100 (advanced)
step_size = 30 #@param {type: "number"}

# Iterations - how many frames to optimize the image. Determines the length of the output video (advanced)
iterations = 50 #@param {type: "number"}


output_path = "/content/output"

social = False

In [ ]:
if image_file == '':
    !pip install git+https://github.com/openai/CLIP.git
    import clip
    import torch

    # Load the open CLIP model
    device = "cuda" if torch.cuda.is_available() else "cpu"
    clip_model, preprocess = clip.load("ViT-B/32", device=device)

    from pathlib import Path

    # Create a folder for the precomputed features
    !mkdir unsplash-dataset

    # Download from Github Releases
    if not Path('unsplash-dataset/photo_ids.csv').exists():
        !wget https://github.com/haltakov/natural-language-image-search/releases/download/1.0.0/photo_ids.csv -O unsplash-dataset/photo_ids.csv

    if not Path('unsplash-dataset/features.npy').exists():
        !wget https://github.com/haltakov/natural-language-image-search/releases/download/1.0.0/features.npy -O unsplash-dataset/features.npy

    import pandas as pd
    import numpy as np

    # Load the photo IDs
    photo_ids = pd.read_csv("unsplash-dataset/photo_ids.csv")
    photo_ids = list(photo_ids['photo_id'])

    # Load the features vectors
    photo_features = np.load("unsplash-dataset/features.npy")

    # Convert features to Tensors: Float32 on CPU and Float16 on GPU
    if device == "cpu":
        photo_features = torch.from_numpy(photo_features).float().to(device)
    else:
        photo_features = torch.from_numpy(photo_features).to(device)

    # Print some statistics
    print(f"Photos loaded: {len(photo_ids)}")


    def encode_search_query(search_query):
        with torch.no_grad():
            # Encode and normalize the search query using CLIP
            text_encoded = clip_model.encode_text(clip.tokenize(search_query).to(device))
            text_encoded /= text_encoded.norm(dim=-1, keepdim=True)

        # Retrieve the feature vector
        return text_encoded


    def find_best_matches(text_features, photo_features, photo_ids, results_count=3):
        # Compute the similarity between the search query and each photo using the Cosine similarity
        similarities = (photo_features @ text_features.T).squeeze(1)

        # Sort the photos by their similarity score
        best_photo_idx = (-similarities).argsort()

        # Return the photo IDs of the best matches
        return [photo_ids[i] for i in best_photo_idx[:results_count]]


    !apt-get install imagemagick
    def search_unslash(search_query, photo_features, photo_ids, results_count=10):
        # Encode the search query
        text_features = encode_search_query(search_query)

        # Find the best matches
        best_photo_ids = find_best_matches(text_features, photo_features, photo_ids, results_count)

        # Selecgt random photo
        best_photo_id = best_photo_ids[np.random.randint(len(best_photo_ids))]
        print(best_photo_id)
        !wget https://unsplash.com/photos/{best_photo_id}/download?w=512 -O /content/photo.jpg
        !convert -blur 0x4 photo.jpg photo.jpg
        return "/content/photo.jpg"

    image_file = search_unslash(text_input, photo_features, photo_ids, 8)   

In [ ]:
# Model to use. ruDALLE was recently finetuned by a russian company called Sber and could give better outputs.
imagemodel = "imagenet" 
 #@param ['imagenet', 'ruDALLE']


# check if christmas and add christmas to prompt
from datetime import datetime

d = datetime.now()

if social and (d.strftime("%m") == "12") and  ((d.strftime("%d") == "24") or  (d.strftime("%d") == "25") or  (d.strftime("%d") == "26")):
    text_input = text_input + "|christmas:3|text:-0.5"

image_prompt = image_file

if len(image_prompt) == 0:
    image_prompt = None

In [ ]:
%cd /content
#@title Upscale images/video frames

black_and_white = False

loaded_upscale_model = False

def upscale(filepath):
  if not super_resolution:
    return
  global loaded_upscale_model
  if not loaded_upscale_model:
    # Clone Real-ESRGAN and enter the Real-ESRGAN
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    %cd /content/Real-ESRGAN
    !git checkout 3338b31f486586bd7f6b20cc2a9fadd5ed192a00
    # Set up the environment
    !pip install git+https://github.com/xinntao/BasicSR.git
    !pip install facexlib
    !pip install gfpgan
    !pip install -r requirements.txt
    !python setup.py develop
    # Download the pre-trained model
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models
    %cd -
    loaded_upscale_model = True 
  
  %cd /content/Real-ESRGAN
  !python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus_anime_6B.pth --input $filepath --netscale 4 --outscale 2 --half --output $output_path
  filepath_out = filepath.replace(".jpg","_out.jpg")
  !mv -v $filepath_out $filepath
  %cd -

In [ ]:
!nvidia-smi

In [ ]:
!mkdir -p $output_path

!git clone https://github.com/openai/CLIP
!git clone https://github.com/CompVis/taming-transformers
#%cd taming-transformers
#!git checkout 2908a53b88478e5812d619b6ac003dbb29b069a0
#%cd -
#!pip install torchvision@https://download.pytorch.org/whl/cu111/torchvision-0.11.1%2Bcu111-cp37-cp37m-linux_x86_64.whl torch torchaudio torchtext
!pip install ftfy regex tqdm omegaconf pytorch-lightning
!pip install kornia
!pip install einops

In [ ]:
from pathlib import Path
!sudo apt install -y aria2

model_mapping = {
    "ruDALLE": { 
        "name": "vqgan_openimages_f16_8192.ckpt",
        "config": "https://cloudflare-ipfs.com/ipfs/QmXey26KJ1S5fc5gtrXbGqdpgc3xvoQiApVYCxzE5uB9D4/vqgan.gumbelf8-sber.model.ckpt.yaml",
        "checkpoint": "https://cloudflare-ipfs.com/ipfs/QmXey26KJ1S5fc5gtrXbGqdpgc3xvoQiApVYCxzE5uB9D4/vqgan.gumbelf8-sber.model.ckpt",
    },
    "imagenet": {
        "name": "vqgan_imagenet_f16_16384.ckpt",
        "config": 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1',
        "checkpoint": 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1',        
    } 
}

selected_vqgan = model_mapping[imagemodel]
vqgan_name = selected_vqgan["name"]

config = selected_vqgan["config"]
checkpoint = selected_vqgan["checkpoint"]

if not Path(vqgan_name+".yaml").exists():
    !aria2c -x 5 --auto-file-renaming=false '{config}' -o {vqgan_name}.yaml
    !aria2c -x 5 --auto-file-renaming=false '{checkpoint}'  -o {vqgan_name}


In [ ]:
import argparse
import math
from pathlib import Path
import sys

sys.path.append('./taming-transformers')

import tensorflow
from IPython import display
from omegaconf import OmegaConf
from PIL import Image
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from taming.models import cond_transformer, vqgan
from tqdm import tqdm
import numpy as np

from CLIP import clip

import kornia.augmentation as K

In [ ]:
def noise_gen(shape):
    n, c, h, w = shape
    noise = torch.zeros([n, c, 1, 1])
    for i in reversed(range(5)):
        h_cur, w_cur = h // 2**i, w // 2**i
        noise = F.interpolate(noise, (h_cur, w_cur), mode='bicubic', align_corners=False)
        noise += torch.randn([n, c, h_cur, w_cur]) / 5
    return noise


def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))


def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()


def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]


def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size

    input = input.view([n * c, 1, h, w])

    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])

    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])

    input = input.view([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)
    

# def replace_grad(fake, real):
#     return fake.detach() - real.detach() + real


class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward

    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)


class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)

    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None

replace_grad = ReplaceGrad.apply

clamp_with_grad = ClampWithGrad.apply
# clamp_with_grad = torch.clamp

def vector_quantize(x, codebook):
    d = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = d.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(d.dtype) @ codebook
    return replace_grad(x_q, x)


class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))

    def forward(self, input):
        
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize((self.embed).unsqueeze(0), dim=2)

        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()


def parse_prompt(prompt):
    vals = prompt.rsplit(':', 2)
    vals = vals + ['', '1', '-inf'][len(vals):]
    try:
        return vals[0], float(vals[1]), float(vals[2])
    except:
        return prompt, 1, float('-inf')

def one_sided_clip_loss(input, target, labels=None, logit_scale=100):
    input_normed = F.normalize(input, dim=-1)
    target_normed = F.normalize(target, dim=-1)
    logits = input_normed @ target_normed.T * logit_scale
    if labels is None:
        labels = torch.arange(len(input), device=logits.device)
    return F.cross_entropy(logits, labels)

class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow

        self.av_pool = nn.AdaptiveAvgPool2d((self.cut_size, self.cut_size))
        self.max_pool = nn.AdaptiveMaxPool2d((self.cut_size, self.cut_size))

    def set_cut_pow(self, cut_pow):
      self.cut_pow = cut_pow

    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        cutouts_full = []
        
        min_size_width = min(sideX, sideY)
        lower_bound = float(self.cut_size/min_size_width)
        
        for ii in range(self.cutn):
            
            
          size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)

          offsetx = torch.randint(0, sideX - size + 1, ())
          offsety = torch.randint(0, sideY - size + 1, ())
          cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
          cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))

        
        cutouts = torch.cat(cutouts, dim=0)

        if args.use_augs:
          cutouts = augs(cutouts)

        if args.noise_fac:
          facs = cutouts.new_empty([cutouts.shape[0], 1, 1, 1]).uniform_(0, args.noise_fac)
          cutouts = cutouts + facs * torch.randn_like(cutouts)
        

        return clamp_with_grad(cutouts, 0, 1)


def load_vqgan_model(config_path, checkpoint_path):
    config = OmegaConf.load(config_path)
    if config.model.target == 'taming.models.vqgan.VQModel':
        model = vqgan.VQModel(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    elif config.model.target == 'taming.models.cond_transformer.Net2NetTransformer':
        parent_model = cond_transformer.Net2NetTransformer(**config.model.params)
        parent_model.eval().requires_grad_(False)
        parent_model.init_from_ckpt(checkpoint_path)
        model = parent_model.first_stage_model
    elif config.model.target == 'taming.models.vqgan.GumbelVQ':
        model = vqgan.GumbelVQ(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    else:
        raise ValueError(f'unknown model type: {config.model.target}')
    del model.loss
    return model

def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)

class TVLoss(nn.Module):
    def forward(self, input):
        input = F.pad(input, (0, 1, 0, 1), 'replicate')
        x_diff = input[..., :-1, 1:] - input[..., :-1, :-1]
        y_diff = input[..., 1:, :-1] - input[..., :-1, :-1]
        diff = x_diff**2 + y_diff**2 + 1e-8
        return diff.mean(dim=1).sqrt().mean()

class GaussianBlur2d(nn.Module):
    def __init__(self, sigma, window=0, mode='reflect', value=0):
        super().__init__()
        self.mode = mode
        self.value = value
        if not window:
            window = max(math.ceil((sigma * 6 + 1) / 2) * 2 - 1, 3)
        if sigma:
            kernel = torch.exp(-(torch.arange(window) - window // 2)**2 / 2 / sigma**2)
            kernel /= kernel.sum()
        else:
            kernel = torch.ones([1])
        self.register_buffer('kernel', kernel)

    def forward(self, input):
        n, c, h, w = input.shape
        input = input.view([n * c, 1, h, w])
        start_pad = (self.kernel.shape[0] - 1) // 2
        end_pad = self.kernel.shape[0] // 2
        input = F.pad(input, (start_pad, end_pad, start_pad, end_pad), self.mode, self.value)
        input = F.conv2d(input, self.kernel[None, None, None, :])
        input = F.conv2d(input, self.kernel[None, None, :, None])
        return input.view([n, c, h, w])

class EMATensor(nn.Module):
    """implmeneted by Katherine Crowson"""
    def __init__(self, tensor, decay):
        super().__init__()
        self.tensor = nn.Parameter(tensor)
        self.register_buffer('biased', torch.zeros_like(tensor))
        self.register_buffer('average', torch.zeros_like(tensor))
        self.decay = decay
        self.register_buffer('accum', torch.tensor(1.))
        self.update()
    
    @torch.no_grad()
    def update(self):
        if not self.training:
            raise RuntimeError('update() should only be called during training')

        self.accum *= self.decay
        self.biased.mul_(self.decay)
        self.biased.add_((1 - self.decay) * self.tensor)
        self.average.copy_(self.biased)
        self.average.div_(1 - self.accum)

    def forward(self):
        if self.training:
            return self.tensor
        return self.average

%mkdir /content/vids

# ARGS

In [ ]:
# scale step size
step_size = step_size / 100
step_size = step_size*step_size

if vqgan_name == 'vqgan_openimages_f16_8192.ckpt':
  step_size=step_size / 20
  
args = argparse.Namespace(
    
    prompts=[t.strip() for t in text_input.split("|")],
    size=[width, height], 
    init_image=image_prompt,
    init_weight= 0.5,

    # clip model settings
    clip_model='ViT-B/32',
    vqgan_config=f'{vqgan_name}.yaml',         
    vqgan_checkpoint=vqgan_name,
    step_size=step_size,
    
    # cutouts / crops
    cutn=32,
    cut_pow=1,
    cut_size=224,

    # display
    display_freq=5,
    seed=None,
    use_augs = True,
    noise_fac= 0.1,

    record_generation=True,

    # noise and other constraints
    use_noise = None,
    constraint_regions = False,#
    
    
    # add noise to embedding
    noise_prompt_weights = None,
    noise_prompt_seeds = [14575],#

    # mse settings
    mse_withzeros = True,
    mse_decay_rate = 50,
    mse_epoches = 10,

    # end itteration
    max_itter = iterations,
)

mse_decay = 0
if args.init_weight:
  mse_decay = args.init_weight / args.mse_epoches

# <AUGMENTATIONS>
augs = nn.Sequential(
    
    K.RandomHorizontalFlip(p=0.5),
    K.RandomAffine(degrees=30, translate=0.1, p=0.8, padding_mode='border'), # padding_mode=2
    K.RandomPerspective(0.2,p=0.4, ),
    K.ColorJitter(hue=0.01, saturation=0.01, p=0.7),

    )

noise = noise_gen([1, 3, args.size[0], args.size[1]])
image = TF.to_pil_image(noise.div(5).add(0.5).clamp(0, 1)[0])
image.save('init3.png')

### Actually do the run...

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print('Using device:', device)
print('using prompts: ', args.prompts)

tv_loss = TVLoss() 

model = load_vqgan_model(args.vqgan_config, args.vqgan_checkpoint).to(device)
perceptor = clip.load(args.clip_model, jit=False)[0].eval().requires_grad_(False).to(device)
mse_weight = args.init_weight

cut_size = args.cut_size
# e_dim = model.quantize.e_dim

if args.vqgan_checkpoint == 'vqgan_openimages_f16_8192.ckpt':
    e_dim = 256
    n_toks = model.quantize.n_embed
    z_min = model.quantize.embed.weight.min(dim=0).values[None, :, None, None]
    z_max = model.quantize.embed.weight.max(dim=0).values[None, :, None, None]
else:
    e_dim = model.quantize.e_dim
    n_toks = model.quantize.n_e
    z_min = model.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
    z_max = model.quantize.embedding.weight.max(dim=0).values[None, :, None, None]


make_cutouts = MakeCutouts(cut_size, args.cutn, cut_pow=args.cut_pow)

f = 2**(model.decoder.num_resolutions - 1)
toksX, toksY = args.size[0] // f, args.size[1] // f

if args.seed is not None:
    torch.manual_seed(args.seed)

if args.init_image:
    pil_image = Image.open(args.init_image).convert('RGB')
    pil_image = pil_image.resize((toksX * 16, toksY * 16), Image.LANCZOS)
    pil_image = TF.to_tensor(pil_image)
    if args.use_noise:
      pil_image = pil_image + args.use_noise * torch.randn_like(pil_image) 
    z, *_ = model.encode(pil_image.to(device).unsqueeze(0) * 2 - 1)

else:
    
    one_hot = F.one_hot(torch.randint(n_toks, [toksY * toksX], device=device), n_toks).float()

    if args.vqgan_checkpoint == 'vqgan_openimages_f16_8192.ckpt':
        z = one_hot @ model.quantize.embed.weight
    else:
        z = one_hot @ model.quantize.embedding.weight
    z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2)

if args.mse_withzeros and not args.init_image:
  z_orig = torch.zeros_like(z)
else:
  z_orig = z.clone()

z.requires_grad = True

opt = optim.Adam([z], lr=args.step_size, weight_decay=0.00000000)

normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                 std=[0.26862954, 0.26130258, 0.27577711])

pMs = []

if args.noise_prompt_weights and args.noise_prompt_seeds:
  for seed, weight in zip(args.noise_prompt_seeds, args.noise_prompt_weights):
    gen = torch.Generator().manual_seed(seed)
    embed = torch.empty([1, perceptor.visual.output_dim]).normal_(generator=gen)
    pMs.append(Prompt(embed, weight).to(device))

for prompt in args.prompts:
    txt, weight, stop = parse_prompt(prompt)
    embed = perceptor.encode_text(clip.tokenize(txt).to(device)).float()
    pMs.append(Prompt(embed, weight, stop).to(device))

def synth_gumbel(z, quantize=True, saturate=True):
    logits = model.quantize.proj(z)
    if quantize:
        one_hot = F.gumbel_softmax(logits, tau=1, hard=True, dim=1)
    else:
        one_hot = F.one_hot(logits.argmax(1), logits.shape[1]).movedim(3, 1).to(logits.dtype)
    z_q = torch.einsum('nchw,cd->ndhw', one_hot, model.quantize.embed.weight)
    return clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

def synth(z, quantize=True, saturate=True):
    out = None
    if args.vqgan_checkpoint == 'vqgan_openimages_f16_8192.ckpt':
       out = synth_gumbel(z, quantize, saturate)
    else:
      if args.constraint_regions:
        z = replace_grad(z, z * z_mask)

      if quantize:
        z_q = vector_quantize(z.movedim(1, 3), model.quantize.embedding.weight).movedim(3, 1)
      else:
        z_q = z.model

      out = clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

    if saturate and not image_prompt:
      progress = i / args.max_itter
      saturation = max(0,min(1,(progress - 0.25) * 2))
      out = transforms.functional.adjust_saturation(out, saturation)
    
    return out

@torch.no_grad()
def checkin(i, losses):
    losses_str = ', '.join(f'{loss.item():g}' for loss in losses)
    tqdm.write(f'i: {i}, loss: {sum(losses).item():g}, losses: {losses_str}')
    out = synth(z, True)# False)

    TF.to_pil_image(out[0].cpu()).save(f'progress.png')   
    #display.display(display.Image('progress.png')) 


def ascend_txt(i):
    global mse_weight

    out = synth(z)
    if args.record_generation:
      with torch.no_grad():
        global vid_index
        out_a = synth(z, True)#, False)
        if vid_index % 5 == 0:
          filename = f'{output_path}/progress_{vid_index:05}.jpg'
          TF.to_pil_image(out_a[0].cpu()).save(filename)
          upscale(filename)
        vid_index += 1

    cutouts = make_cutouts(out)
    cutouts = resample(cutouts, (perceptor.visual.input_resolution, perceptor.visual.input_resolution))


    iii = perceptor.encode_image(normalize(cutouts)).float()

    result = []

    if args.init_weight:
        
        global z_orig
        
        result.append(F.mse_loss(z, z_orig) * mse_weight / 2)
        # result.append(F.mse_loss(z, z_orig) * ((1/torch.tensor((i)*2 + 1))*mse_weight) / 2)

        with torch.no_grad():
          if i > 0 and i%args.mse_decay_rate==0 and i <= args.mse_decay_rate*args.mse_epoches:

            if mse_weight - mse_decay > 0 and mse_weight - mse_decay >= mse_decay:
              mse_weight = mse_weight - mse_decay
              print(f"updated mse weight: {mse_weight}")
            else:
              mse_weight = 0
              print(f"updated mse weight: {mse_weight}")

    for prompt in pMs:
        result.append(prompt(iii))

    return result

vid_index = 0
def train(i):
    
    opt.zero_grad()
    lossAll = ascend_txt(i)

    if i % args.display_freq == 0:
        checkin(i, lossAll)
    
    loss = sum(lossAll)

    loss.backward()
    opt.step()

i = 0
try:
    for i in tqdm(range(args.max_itter)):

        train(i)

        if i > 0 and i%args.mse_decay_rate==0 and i <= args.mse_decay_rate * args.mse_epoches:
          
          opt = optim.Adam([z], lr=args.step_size, weight_decay=0.00000000)

except KeyboardInterrupt:
    pass


# create video

In [ ]:
out_file=output_path+"/video.mp4"

!mkdir -p /tmp/ffmpeg
!cp $output_path/*.jpg /tmp/ffmpeg
last_frame=!ls -w1 -t /tmp/ffmpeg/*.jpg | head -1
last_frame = last_frame[0]

# Copy last frame to start and duplicate at end so it sticks around longer
end_still_seconds = 4
for i in range(end_still_seconds * 10):
  pad_file = f"/tmp/ffmpeg/zzzz_pad_{i:05}.jpg"
  !cp -v $last_frame $pad_file

!ffmpeg  -r 3 -i /tmp/ffmpeg/%*.jpg -y -c:v libx264 -r 15 /tmp/vid_no_audio.mp4
!ffmpeg -i /tmp/vid_no_audio.mp4 -f lavfi -i anullsrc -c:v copy -c:a aac -shortest -y "$out_file"

print("Written", out_file)
!sleep 2
!rm -r /tmp/ffmpeg

'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00005.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00006.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00007.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00008.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00009.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00010.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00011.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00012.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00013.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00014.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00015.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00016.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00017.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -> '/tmp/ffmpeg/zzzz_pad_00018.jpg'
'/tmp/ffmpeg/progress_00555.jpg' -

In [ ]:
import os.path
if not os.path.exists(out_file):
  raise Exception("Expected output file does not exist.")